In [58]:
import os

In [59]:
%pwd

'C:\\Users\\Tanaya\\Downloads\\Projects\\End-to-End-Machine-Learning-Project-with-MLFlow'

In [60]:
os.chdir("C:/Users/Tanaya/Downloads/Projects/End-to-End-Machine-Learning-Project-with-MLFlow")


In [61]:
%pwd

'C:\\Users\\Tanaya\\Downloads\\Projects\\End-to-End-Machine-Learning-Project-with-MLFlow'

In [62]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tanayatipre8/End-to-End-Machine-Learning-Project-with-MLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tanayatipre8"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e65a37b4384f5f8f1cd181fc2223507eb0c58591"

In [63]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [64]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

In [65]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tanayatipre8/End-to-End-Machine-Learning-Project-with-MLFlow.mlflow"

        )

        return model_evaluation_config

In [66]:
import os
import pandas as pd
from MLProject import logger
from sklearn.linear_model import ElasticNet
import joblib
import mlflow.sklearn
import mlflow
from urllib.parse import urlparse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from MLProject.utils.common import save_json


In [67]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae= mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local

            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")



In [68]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e 

[2025-07-15 00:41:38,091: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-15 00:41:38,116: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-15 00:41:38,124: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-07-15 00:41:38,126: INFO: common: created directory at artifacts]
[2025-07-15 00:41:38,127: INFO: common: created directory at artifacts/model_evaluation]
[2025-07-15 00:41:38,556: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/07/15 00:41:54 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
